# Movie data preprocessing
## movies와 credits에서 프로젝트에 사용되지 않는 컬럼을 버린다
### budget, homepage, original_language, original_title, production_countries, release_date, revenue, spoken_language, status

## movies와 credits dataframe을 merge한다.
## crew 컬럼에서 director 정보를 뽑아내서 새로운 DataFrame으로 만든다.
### director1 ~ director 종류수

In [1]:
import pandas as pd
import numpy as npc

movies_df = pd.read_csv("data/tmdb_5000_movies.csv")
credits_df = pd.read_csv("data/tmdb_5000_credits.csv")

# drop columns
# budget, homepage, original_language, original_title, production_countries, release_date,
# revenue, spoken_languages, status
movies_df = movies_df.drop(columns=['budget', 'homepage', 'original_language', 'original_title', 'production_countries',
                                    'release_date', 'revenue', 'spoken_languages', 'status', 'runtime', 'tagline'])

credits_df = credits_df.drop(columns=['title'])
credits_df.columns=['id', 'cast', 'crew']

In [2]:
total_df = movies_df.join(credits_df.set_index('id'), on='id')

In [3]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 11 columns):
genres                  4803 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
cast                    4803 non-null object
crew                    4803 non-null object
dtypes: float64(2), int64(2), object(7)
memory usage: 412.8+ KB


In [4]:
import json

json_columns = ['genres', 'keywords', 'cast', 'crew', 'production_companies']
for column in json_columns:
    total_df[column] = total_df[column].apply(json.loads)


In [5]:
count=0
for title, cast_list in zip(total_df['title'], total_df["crew"]):
    if not cast_list:
        print(title)
        count+=1
print(count)

Flying By
Running Forever
Boynton Beach Club
Sharkskin
The Book of Mormon Movie, Volume 1: The Journey
Hum To Mohabbat Karega
Roadside Romeo
Me You and Five Bucks
Down & Out With The Dolls
Crowsnest
Sex With Strangers
The Helix... Loaded
Karachi se Lahore
Harrison Montgomery
Light from the Darkroom
America Is Still the Place
The Little Ponderosa Zoo
Diamond Ruff
Rise of the Entrepreneur: The Search for a Better Way
I Want Your Money
Fabled
Death Calls
Amidst the Devil's Wings
Teeth and Blood
UnDivided
Little Big Top
Short Cut to Nirvana: Kumbh Mela
The Blood of My Brother: A Story of Death in Iraq
28


In [6]:
director_list = list()
director_set = set()
for title, crew_list in zip(total_df['title'], total_df['crew']):
    temp = list()
    for one_crew_dict in crew_list:
        if one_crew_dict['job'] == 'Director':
            temp.append(one_crew_dict['name'])
            director_set.add(one_crew_dict['name'])
    director_list.append(temp)
#total_df['director'] = directors

In [7]:
director_list
director_set
director_dict = dict()
director_dict = dict.fromkeys(director_set)
count = 0
for key in director_dict.keys():
    director_dict[key] = count
    count+=1
director_rev_dict = {v:k for k,v in director_dict.items()}
print(count)

2577


In [8]:
director_vector = list()
for directors in director_list:
    temp_director = [0]*2577
    temp_name = list()
    for director in directors:
        temp_director[director_dict[director]] = 1
    director_vector.append(temp_director)

print(len(director_vector))


4803


In [9]:
import numpy as np

column_names = list()
count = 0
while True:
    if count >= 2577:
        break
    name = 'Director' + str(count)
    column_names.append(name)
    count += 1
Directors = pd.DataFrame(director_vector, columns=column_names)


In [10]:
#Directors['names'] = director_list

In [11]:
director_count = list()
for item in director_list:
    director_count.append(len(item))
#Directors['count'] = director_count

In [12]:
#Directors['title'] = total_df['title']

In [13]:
#title_director = pd.DataFrame({'title' : Directors['title'], 'count' : Directors['count']})

In [14]:
#title_director.head(10)

In [15]:
#title_director.plot(x='title', y='count')

# Production Companies vectorize
## production company 의 종류 개수 : 5017
## 한 영화가 가진 가장 많은 production company의 종류 개수 : 26
## 한 영화가 가진 가장 적은 production company의 종류 개수 : 0

In [16]:
pro_com_set = set()
com_list_list = list()
for company_list in total_df['production_companies']:
    temp_com_list = list()
    for company in company_list:
        pro_com_set.add(company['name'])
        temp_com_list.append(company['name'])
    com_list_list.append(temp_com_list)

pro_com_vectors = list()
pro_com_dict = dict()
pro_com_dict = dict.fromkeys(pro_com_set)
count = 0
for keyitem in pro_com_dict.keys():
    pro_com_dict[keyitem] = count
    count += 1

for com_list in com_list_list:
    temp_vec = [0]*5017
    for com in com_list:
        temp_vec[pro_com_dict[com]] = 1
    pro_com_vectors.append(temp_vec)

production_com_columns = list()
for i in range(5017):
    temp_name = 'company' + str(i)
    production_com_columns.append(temp_name)

Production_Companies = pd.DataFrame(pro_com_vectors, columns = production_com_columns)
#Production_Companies['title'] = total_df['title']

In [17]:
# 한 영화가 가진 가장 많은 production company의 종류 개수 : 26
max_len = 0
for com_list in com_list_list:
    if len(com_list) > max_len:
        max_len = len(com_list)
print(max_len)

26


In [18]:
# 한 영화가 가진 가장 적은 production company의 종류 개수 : 0
min_len = 9999
for com_list in com_list_list:
    if len(com_list) < min_len:
        min_len = len(com_list)
print(min_len)

0


# Genre vectorize
## genre 의 종류 개수 : 20
## 한 영화가 가진 가장 많은 genre의 종류 개수 : 7
## 한 영화가 가진 가장 적은 genre의 종류 개수 : 0

In [19]:
genre_set = set()
genre_list_list = list()

for genre_list in total_df['genres']:
    temp_genre = list()
    for genre in genre_list:
        genre_set.add(genre['name'])
        temp_genre.append(genre['name'])
    genre_list_list.append(temp_genre)
print(len(genre_set))
print(genre_set)

20
{'Western', 'Family', 'Adventure', 'Animation', 'Mystery', 'History', 'Foreign', 'War', 'Drama', 'Comedy', 'Music', 'Action', 'Crime', 'Science Fiction', 'Thriller', 'Documentary', 'Horror', 'Romance', 'TV Movie', 'Fantasy'}


In [20]:
genre_dict = dict()
genre_dict = dict.fromkeys(genre_set)
count = 0
for key_item in genre_dict.keys():
    genre_dict[key_item] = count
    count += 1
genre_vector = list()
for genre_list in genre_list_list:
    temp_genre_vec = [0] * 20
    for genre in genre_list:
        temp_genre_vec[genre_dict[genre]] = 1
    genre_vector.append(temp_genre_vec)
genre_cols = list()
for i in range(20):
    temp_genre = 'genre' + str(i)
    genre_cols.append(temp_genre)

Genre = pd.DataFrame(genre_vector, columns=genre_cols)

In [21]:
Genre.head(10)

,genre0,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10,genre11,genre12,genre13,genre14,genre15,genre16,genre17,genre18,genre19
0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
6,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
8,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [22]:
# 한 영화가 가진 가장 많은 genre의 종류 개수 : 7
max_len = 0
for genre_list in genre_list_list:
    if len(genre_list) > max_len:
        max_len = len(genre_list)
print(max_len)

7


In [23]:
# 한 영화가 가진 가장 적은 genre의 종류 개수 : 0
min_len = 9999
for genre_list in genre_list_list:
    if len(genre_list) < min_len:
        min_len = len(genre_list)
print(min_len)

0


# Overview vectorize - tf-idf를 사용.
# sparse matrix를 기반으로 cosine similarity를 계산
1. sci-kit learn의 TfidfVectorizer 모듈을 사용하여 row : movie, column : 단어의 빈도수인 행렬을 만든다. 

2. sci-kit learn의 cosine_similarities()보다 속도면에서 우수한 linear_kernel()을 사용하여 cosine similarity score를 계산

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')

total_df['overview'] = total_df['overview'].fillna('')

# construct TF-IDF matrix by fitting and transforming the overview data
tfidf_matrix = tfidf.fit_transform(total_df['overview'])

# output the shape of tfidf_matrix
tfidf_matrix.shape

(4803, 20978)

In [25]:
import scipy
# overview 학습의 결과를 unpack 해보았다.
M1 = scipy.sparse.diags(tfidf_matrix.A.ravel()).tocsr()
M1

<100757334x100757334 sparse matrix of type '<class 'numpy.float64'>'
	with 125840 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(total_df.index, index=total_df['title']).drop_duplicates()

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, target_count=10, cosine_sim=cosine_similarity):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:1 + target_count] # except for itself
    movies_indices = [i[0] for i in sim_scores]
    for index, score in zip(movies_indices, sim_scores):
        print("index:", score[0],"  ","score:", score[1], "   ", total_df['title'].iloc[index])
    #return total_df['title'].iloc[movies_indices]

In [27]:
get_recommendations('Avatar')

index: 3604    score: 0.1868100105621205     Apollo 18
index: 2130    score: 0.17015105119162663     The American
index: 634    score: 0.13230696630317224     The Matrix
index: 1341    score: 0.12393264590871877     The Inhabited Island
index: 529    score: 0.12248959164893025     Tears of the Sun
index: 1610    score: 0.1046990303761563     Hanna
index: 311    score: 0.10091388184065114     The Adventures of Pluto Nash
index: 847    score: 0.09799623121705055     Semi-Pro
index: 775    score: 0.09447938436890334     Supernova
index: 2628    score: 0.09313854469530522     Blood and Chocolate


In [28]:
cast_all = dict()
for cast_group in total_df['cast']:
    for cast in cast_group:
        if cast['id'] in cast_all:
            cast_all[cast['id']][1] += 1
        else:
            cast_all[cast['id']] = [cast['name'], 1]

In [29]:
len(cast_all)

54588

In [30]:
cast_freq = list(cast_all.values())
cast_freq.sort(reverse=True, key=lambda x: (x[1]))

In [31]:
cast_freq

[['Samuel L. Jackson', 67],
 ['Robert De Niro', 57],
 ['Bruce Willis', 51],
 ['Matt Damon', 48],
 ['Morgan Freeman', 46],
 ['Steve Buscemi', 43],
 ['Liam Neeson', 41],
 ['Johnny Depp', 40],
 ['Owen Wilson', 40],
 ['John Goodman', 39],
 ['Nicolas Cage', 39],
 ['Alec Baldwin', 39],
 ['Willem Dafoe', 38],
 ['Jim Broadbent', 38],
 ['Stanley Tucci', 38],
 ['Brad Pitt', 38],
 ['Paul Giamatti', 37],
 ['Will Ferrell', 37],
 ['Richard Jenkins', 36],
 ['Susan Sarandon', 36],
 ['J.K. Simmons', 35],
 ['Bill Murray', 35],
 ['Christopher Walken', 35],
 ['Keith David', 35],
 ['Bill Hader', 34],
 ['Jack Black', 34],
 ['Paul Rudd', 34],
 ['Ben Kingsley', 34],
 ['George Clooney', 34],
 ['Anthony Hopkins', 34],
 ['Robin Williams', 34],
 ['Justin Long', 34],
 ['David Koechner', 34],
 ['Sigourney Weaver', 33],
 ['Laurence Fishburne', 33],
 ['Frank Welker', 33],
 ['Tom Cruise', 33],
 ['Ben Stiller', 33],
 ['Brian Cox', 33],
 ['William H. Macy', 33],
 ['Julia Roberts', 33],
 ['Tom Hanks', 32],
 ['Arnold Schw

In [32]:
cast2vec_dict = dict()
count = 0;
for cast_id in cast_all.keys():
    cast2vec_dict[cast_id] = count
    count += 1

cast_vectors = list()
for cast_group in total_df['cast']:
    cast_vector = [0] * len(cast_all)
    for cast in cast_group:
        cast_vector[cast2vec_dict[cast['id']]] = 1
    cast_vectors.append(cast_vector)

In [33]:
len(cast_vectors[0])

54588

In [34]:
cast_col = list()
count = 0
for ele in cast_vectors[0]:
    name = 'cast' + str(count)
    count += 1
    cast_col.append(name)

In [35]:
Cast = pd.DataFrame(cast_vectors, columns=cast_col)

In [36]:
Cast.shape

(4803, 54588)

# Director, Genre, Production Companies, Cast 벡터화 완료
## 각 벡터화의 결과를 DataFrame 형태로 저장함
## 각 DataFrame의 shape를 확인

In [37]:
print('-'*62)
print("-"*20 + "Vectorization Complete" + "-"*20)
print("Director shape : ", Directors.shape)
print("Genre shape", Genre.shape)
print("Production Companies shape : ", Production_Companies.shape)
print("Cast shape : ", Cast.shape)
print('-'*62)

--------------------------------------------------------------
--------------------Vectorization Complete--------------------
Director shape :  (4803, 2577)
Genre shape (4803, 20)
Production Companies shape :  (4803, 5017)
Cast shape :  (4803, 54588)
--------------------------------------------------------------


# cosine similarity 를 구하기 위해 numpy array 를 sparse matrix으로 변환

In [38]:
from scipy import sparse

Directors_sparse = sparse.csr_matrix(Directors.values)
Genre_sparse = sparse.csr_matrix(Genre.values)
Procom_sparse = sparse.csr_matrix(Production_Companies.values)
Cast_sparse = sparse.csr_matrix(Cast.values)

# 영화의 수가 4800여개뿐이므로 감독 데이터만으로는 상위 10개의 유사도를 갖는 영화들의 목록을 정확히 얻을 수 없다는 것을 알 수 있다.

In [39]:
cosine_sim_directors = linear_kernel(Directors_sparse, Directors_sparse)
get_recommendations('The Dark Knight Rises',10, cosine_sim_directors)

index: 65    score: 1.0     The Dark Knight
index: 95    score: 1.0     Interstellar
index: 96    score: 1.0     Inception
index: 119    score: 1.0     Batman Begins
index: 1033    score: 1.0     Insomnia
index: 1196    score: 1.0     The Prestige
index: 3573    score: 1.0     Memento
index: 0    score: 0.0     Avatar
index: 1    score: 0.0     Pirates of the Caribbean: At World's End
index: 2    score: 0.0     Spectre


In [40]:
cosine_sim_genre = linear_kernel(Genre_sparse, Genre_sparse)
get_recommendations('The Dark Knight Rises',10, cosine_sim_genre)

index: 65    score: 4.0     The Dark Knight
index: 405    score: 4.0     The Fast and the Furious: Tokyo Drift
index: 590    score: 4.0     The Siege
index: 629    score: 4.0     Need for Speed
index: 724    score: 4.0     Man on Fire
index: 741    score: 4.0     Shooter
index: 747    score: 4.0     Gangster Squad
index: 761    score: 4.0     Righteous Kill
index: 762    score: 4.0     Mercury Rising
index: 767    score: 4.0     xXx: State of the Union


In [41]:
cosine_sim_procom = linear_kernel(Procom_sparse, Procom_sparse)
get_recommendations('The Dark Knight Rises',10, cosine_sim_procom)

index: 14    score: 4.0     Man of Steel
index: 65    score: 4.0     The Dark Knight
index: 119    score: 4.0     Batman Begins
index: 95    score: 3.0     Interstellar
index: 96    score: 3.0     Inception
index: 9    score: 2.0     Batman v Superman: Dawn of Justice
index: 10    score: 2.0     Superman Returns
index: 41    score: 2.0     Green Lantern
index: 48    score: 2.0     Jack the Giant Slayer
index: 51    score: 2.0     Pacific Rim


In [42]:
cosine_sim_cast = linear_kernel(Cast_sparse, Cast_sparse)
get_recommendations('The Dark Knight Rises',10, cosine_sim_cast)

index: 65    score: 7.0     The Dark Knight
index: 96    score: 7.0     Inception
index: 119    score: 7.0     Batman Begins
index: 298    score: 7.0     The Wolf of Wall Street
index: 95    score: 6.0     Interstellar
index: 2060    score: 6.0     Out of the Furnace
index: 1424    score: 5.0     Concussion
index: 16    score: 4.0     The Avengers
index: 1196    score: 4.0     The Prestige
index: 27    score: 3.0     Battleship


In [43]:
def get_cast_by_index(idx):
    cast = total_df.values[idx][9]
    return [actor_data['name'] for actor_data in cast]
def get_cast_by_title(title):
    return get_cast_by_index(indices[title])

In [44]:
battleship_cast = get_cast_by_index(27)
knight_cast = get_cast_by_index(3)

for bat in battleship_cast:
    for kni in knight_cast:
        if bat == kni:
            print(bat)

Liam Neeson
James Rawlings
Josh Pence


In [45]:
from operator import add

def total_recommendation(title, target_count=10):
    idx = indices[title]
    # overview score
    cos_sim_overview = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores_overview = list(enumerate(cos_sim_overview[idx]))
    #sim_scores_overview = list(map(lambda x[1]:x[1]*10000, sim_scores_overview))
    
    # Director score
    cos_sim_director = linear_kernel(Directors_sparse, Directors_sparse)
    sim_scores_director = list(enumerate(cos_sim_director[idx]))
    
    # Production Companies score
    cos_sim_procom = linear_kernel(Procom_sparse, Procom_sparse)
    sim_scores_procom = list(enumerate(cos_sim_procom[idx]))
    
    # Genre score
    cos_sim_genre = linear_kernel(Genre_sparse, Genre_sparse)
    sim_scores_genre = list(enumerate(cos_sim_genre[idx]))
    
    # Cast socre
    cos_sim_cast = linear_kernel(Cast_sparse, Cast_sparse)
    sim_scores_cast = list(enumerate(cos_sim_cast[idx]))
    
    #total_scores = [sum(x[0]) for x in zip(sim_scores_overview, sim_scores_director, sim_scores_procom, sim_scores_genre, sim_scores_cast)]
    
    
    """
    sim_scores_overview = sorted(sim_scores_overview, key=lambda x : x[1], reverse = True)
    sim_scores_overview = sim_scores_overview[1:1 + target_count] # except for itself
    """
    total_scores = list()
    for x in zip(sim_scores_overview, sim_scores_director, sim_scores_procom, sim_scores_genre, sim_scores_cast):
        total_score = 0
        for idx, score in enumerate(x):
            if idx == 0:
                total_score += 100*score[1]
            total_score += score[1]
        total_scores.append(total_score)
        
    total_result = sorted(enumerate(total_scores), key=lambda x : x[1], reverse = True)
    total_result = total_result[1:1+target_count]
    movies_indicies = [i[0] for i in total_result]
    
    recommendation_result = list()
    
    for title, ind, score in  zip(total_df['title'].iloc[movies_indicies], movies_indicies, total_result):
        recommendation_result.append([ind, score[1], title])
    
    return recommendation_result

    """
    movies_indices = [i[0] for i in sim_scores]
    for index, score in zip(movies_indices, sim_scores):
        print("index:", score[0],"  ","score:", score[1], "   ", total_df['title'].iloc[index])
    #return total_df['title'].iloc[movies_indices]
    """

In [46]:
total_recommendation("The Avengers")

[[7, 31.783807514395612, 'Avengers: Age of Ultron'],
 [85, 18.86850064026152, 'Captain America: The Winter Soldier'],
 [3144, 13.401932420447402, 'Plastic'],
 [1715, 13.148903861028042, 'Timecop'],
 [26, 12.92263364135301, 'Captain America: Civil War'],
 [129, 12.759879877796381, 'Thor'],
 [31, 12.322123867726315, 'Iron Man 3'],
 [2136, 12.316601667911371, 'Team America: World Police'],
 [4124, 11.86045062897238, 'This Thing of Ours'],
 [79, 11.771264278951355, 'Iron Man 2']]